In [1]:
import anndata as ad
import zarr
import s3fs

In [2]:
s3 = s3fs.S3FileSystem(anon=True)

In [3]:
s3.ls("s3://tmp-census-zarr/v1/homo_sapiens")

['tmp-census-zarr/v1/homo_sapiens/0041b9c3-6a49-4bf7-8514-9bc7190067a7.zarr',
 'tmp-census-zarr/v1/homo_sapiens/00476f9f-ebc1-4b72-b541-32f912ce36ea.zarr',
 'tmp-census-zarr/v1/homo_sapiens/00e5dedd-b9b7-43be-8c28-b0e5c6414a62.zarr',
 'tmp-census-zarr/v1/homo_sapiens/00ff600e-6e2e-4d76-846f-0eec4f0ae417.zarr',
 'tmp-census-zarr/v1/homo_sapiens/01209dce-3575-4bed-b1df-129f57fbc031.zarr',
 'tmp-census-zarr/v1/homo_sapiens/0129dbd9-a7d3-4f6b-96b9-1da155a93748.zarr',
 'tmp-census-zarr/v1/homo_sapiens/015c230d-650c-4527-870d-8a805849a382.zarr',
 'tmp-census-zarr/v1/homo_sapiens/019c7af2-c827-4454-9970-44d5e39ce068.zarr',
 'tmp-census-zarr/v1/homo_sapiens/01ad3cd7-3929-4654-84c0-6db05bd5fd59.zarr',
 'tmp-census-zarr/v1/homo_sapiens/01c93cf6-b695-4e30-a26e-121ae8b16a9e.zarr',
 'tmp-census-zarr/v1/homo_sapiens/02792605-4760-4023-82ad-40fc4458a5db.zarr',
 'tmp-census-zarr/v1/homo_sapiens/03181d87-4769-41e7-8c39-d9a81835f0d2.zarr',
 'tmp-census-zarr/v1/homo_sapiens/0374f03c-62e2-4859-8a14-acb00b

In [4]:
TEST_ANNDATA_URI = "s3://tmp-census-zarr/v1/homo_sapiens/0041b9c3-6a49-4bf7-8514-9bc7190067a7.zarr"

In [5]:
from vitessce import (
    VitessceWidget,
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)

In [6]:
from anndata.experimental import read_elem

In [7]:
z = zarr.open(TEST_ANNDATA_URI)

In [8]:
read_elem(z["/"])

AnnData object with n_obs × n_vars = 9424 × 60530
    obs: 'soma_joinid', 'dataset_id', 'assay', 'assay_ontology_term_id', 'cell_type', 'cell_type_ontology_term_id', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'is_primary_data', 'observation_joinid', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'tissue_general', 'tissue_general_ontology_term_id', 'raw_sum', 'nnz', 'raw_mean_nnz', 'raw_variance_nnz', 'n_measured_vars'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_length', 'nnz', 'n_measured_obs'

In [9]:
vc = VitessceConfig(schema_version="1.0.15")
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        adata_store=z,
        # obs_embedding_paths=["obsm/X_umap"],
        # obs_embedding_names=["UMAP"],
        obs_set_paths=["obs/cell_type"],
        obs_set_names=["cell_type"],
        obs_feature_matrix_path="X",
        # feature_filter_path="var/top_highly_variable"
))
# scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
layout = vc.layout(((cell_sets / genes)) / heatmap)

In [11]:
layout.widget()

VitessceWidget(config={'version': '1.0.15', 'name': '', 'description': '', 'datasets': [{'uid': 'A', 'name': '…

In [15]:
vc.web_app()

'http://vitessce.io/#?theme=light&url=data:,%7B%22version%22%3A+%221.0.15%22%2C+%22name%22%3A+%22%22%2C+%22description%22%3A+%22%22%2C+%22datasets%22%3A+%5B%7B%22uid%22%3A+%22A%22%2C+%22name%22%3A+%22Brain%22%2C+%22files%22%3A+%5B%7B%22fileType%22%3A+%22anndata.zarr%22%2C+%22url%22%3A+%22http%3A%2F%2Flocalhost%3A8000%2FA%2F0%2F84fc3ccb-d199-4d8d-a29a-1da0ec6a9fab.adata.zarr%22%2C+%22options%22%3A+%7B%22obsSets%22%3A+%5B%7B%22name%22%3A+%22cell_type%22%2C+%22path%22%3A+%22obs%2Fcell_type%22%7D%5D%2C+%22obsFeatureMatrix%22%3A+%7B%22path%22%3A+%22X%22%7D%7D%7D%5D%7D%5D%2C+%22coordinationSpace%22%3A+%7B%22dataset%22%3A+%7B%22A%22%3A+%22A%22%7D%7D%2C+%22layout%22%3A+%5B%7B%22component%22%3A+%22obsSets%22%2C+%22coordinationScopes%22%3A+%7B%22dataset%22%3A+%22A%22%7D%2C+%22x%22%3A+0%2C+%22y%22%3A+0.0%2C+%22w%22%3A+12%2C+%22h%22%3A+3.0%7D%2C+%7B%22component%22%3A+%22featureList%22%2C+%22coordinationScopes%22%3A+%7B%22dataset%22%3A+%22A%22%7D%2C+%22x%22%3A+0%2C+%22y%22%3A+3.0%2C+%22w%22%3A+12%2